In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scs

Exploratory hypothesis testing of the influence of a new landing page vs old landing page on user click through rate on data from Etsy

In [4]:
df = pd.read_csv('experiment.csv')
country = pd.read_csv('country.csv')

In [5]:
df.head()

,user_id,ts,ab,landing_page,converted
0,4040615247,1.356998e+09,treatment,new_page,0
1,4365389205,1.356998e+09,treatment,new_page,0
2,4256174578,1.356998e+09,treatment,new_page,0
3,8122359922,1.356998e+09,control,old_page,0
4,6077269891,1.356998e+09,control,old_page,0


In [6]:
country.head()

,user_id,country
0,9160993935,UK
1,5879439034,UK
2,8915383273,UK
3,2917824565,US
4,3980216975,UK
